In [ ]:
#The premise is to find the item categories mix to obtain a certain target profit

In [1]:
#Import necesary librarie
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [2]:
#Load the data
dataset = pd.read_csv("Sales.csv")

In [3]:
#Clean Sales Data

In [4]:
#Determine what needs and what to drop
dataset.head()
dataset.columns
dataset = dataset.drop(axis = 1, columns = ['OrderDateKey', 'DueDateKey', 'ShipDateKey',
       'ResellerKey', 'EmployeeKey', 'PromotionKey', 'CurrencyKey',
       'SalesTerritoryKey',
       'RevisionNumber', 
       'UnitPriceDiscountPct', 'DiscountAmount', 'ProductStandardCost',
        'SalesAmount', 'TaxAmt', 'Freight',
       'CarrierTrackingNumber', 'CustomerPONumber', 'DueDate',
       'ShipDate'])

In [5]:
#Renaming columns
dataset.rename(columns = {"ProductKey":"Item No","SalesOrderNumber":"Order No", 
                       "SalesOrderLineNumber":"Line No","OrderQuantity":"Quantity", 
                       "UnitPrice":"Unit Price" ,  "ExtendedAmount":"Amount", 
                       "TotalProductCost":"Total Cost", 'OrderDate': "Date"}, inplace = True)
dataset.head()

,Amount,Categories,Date,Quantity,Item No,Line No,Order No,Total Cost,Unit Price
0,1466.01,Bikes,2013-07-31 00:00:00,1,372,3,SO63155,1554.9479,1466.01
1,5864.04,Bikes,2013-07-31 00:00:00,4,374,18,SO63155,6219.7916,1466.01
2,1466.01,Bikes,2013-07-31 00:00:00,1,376,7,SO63155,1554.9479,1466.01
3,1466.01,Bikes,2013-07-31 00:00:00,1,378,9,SO63155,1554.9479,1466.01
4,4398.03,Bikes,2013-07-31 00:00:00,3,380,12,SO63155,4664.8437,1466.01


In [6]:
#Convert Date column to date format
dataset.dtypes
dataset['Date'] = dataset['Date'].str[:10]
dataset.head()

,Amount,Categories,Date,Quantity,Item No,Line No,Order No,Total Cost,Unit Price
0,1466.01,Bikes,2013-07-31,1,372,3,SO63155,1554.9479,1466.01
1,5864.04,Bikes,2013-07-31,4,374,18,SO63155,6219.7916,1466.01
2,1466.01,Bikes,2013-07-31,1,376,7,SO63155,1554.9479,1466.01
3,1466.01,Bikes,2013-07-31,1,378,9,SO63155,1554.9479,1466.01
4,4398.03,Bikes,2013-07-31,3,380,12,SO63155,4664.8437,1466.01


In [7]:
#Creating the profit column
dataset['Profit'] = (dataset['Unit Price'] - (dataset["Total Cost"] / dataset["Quantity"])).round(2)
dataset['Unit Cost'] = (dataset["Total Cost"] / dataset["Quantity"]).round(2)
dataset.head()

,Amount,Categories,Date,Quantity,Item No,Line No,Order No,Total Cost,Unit Price,Profit,Unit Cost
0,1466.01,Bikes,2013-07-31,1,372,3,SO63155,1554.9479,1466.01,-88.94,1554.95
1,5864.04,Bikes,2013-07-31,4,374,18,SO63155,6219.7916,1466.01,-88.94,1554.95
2,1466.01,Bikes,2013-07-31,1,376,7,SO63155,1554.9479,1466.01,-88.94,1554.95
3,1466.01,Bikes,2013-07-31,1,378,9,SO63155,1554.9479,1466.01,-88.94,1554.95
4,4398.03,Bikes,2013-07-31,3,380,12,SO63155,4664.8437,1466.01,-88.94,1554.95


In [8]:
#Pivot relevant columns in order to prepare the data for prediction
dataset = dataset.pivot_table(columns = 'Categories', values = 'Unit Cost',  index=['Order No','Date','Item No', 'Profit' ], fill_value=0)
dataset.reset_index(inplace=True)
dataset

Categories,Order No,Date,Item No,Profit,Accessories,Bikes,Clothing,Components
0,SO43659,2010-12-29,218,2.30,0.00,0.00,3.40,0.0
1,SO43659,2010-12-29,220,8.16,12.03,0.00,0.00,0.0
2,SO43659,2010-12-29,223,-0.52,0.00,0.00,5.71,0.0
3,SO43659,2010-12-29,229,-2.88,0.00,0.00,31.72,0.0
4,SO43659,2010-12-29,235,-2.88,0.00,0.00,31.72,0.0
...,...,...,...,...,...,...,...,...
29994,SO71950,2013-11-29,359,125.01,0.00,1251.98,0.00,0.0
29995,SO71950,2013-11-29,474,15.82,0.00,0.00,26.18,0.0
29996,SO71950,2013-11-29,476,15.82,0.00,0.00,26.18,0.0
29997,SO71951,2013-11-29,363,125.01,0.00,1251.98,0.00,0.0


In [9]:
# Separating the data into a training set and a testing set
X = dataset [['Accessories','Bikes','Clothing','Components']]
y = dataset ['Profit']
X_train,X_test,y_train,y_test = train_test_split(X,y)

In [10]:
# Setting up de profit prediction
lin = LinearRegression()
lin.fit(X_train,y_train)
y_pred = lin.predict(X_test)

In [11]:
#Create coefficients necesary for Power BI
coef = lin.coef_
components = pd.DataFrame(zip(X.columns,coef), columns = ["component","value"])
components = components.append({"component": "intercept", "value": lin.intercept_}, ignore_index = True)